## Problem
$$
\frac{A}{10B + C} + \frac{d}{10E + F} + \frac{G}{10H + I} = 1
$$

$$
A, B, I \in \{1,...,9\}
$$

This problem can be generalised to 

$$
\sum^n_{i=1} \frac{a_i}{10b_i + c_i} = 1
$$

Where each digit appears between 1 and $\lceil\frac{n}{3}\rceil$ times

## Fomulation

Number of times $\frac{i}{10j + k}$ is used in the expression is represented by:
$$
x_{ijk} \in \mathbb{I}
$$

$$
i, j, k \in \{1, ..., 9\}
$$

Number of times digit d appears in (i,j,k)
$$
\delta(d, (i,j,k))
$$


The problem is thus:
$$
\sum_{ijk} x_{ijk} (\frac{i}{10j + k}) = 1
$$

The number of times each digit appears constraint
$$
1 \leq \sum_{ijk} \delta(d, (i,j,k))  x_{ijk} \leq \lceil\frac{n}{3}\rceil
$$

In [8]:
import gurobipy as gp
import math
import fractions

n = 20
# Digits
D = range(1, 10)

m = gp.Model()

X = {
    (i, j, k):
    m.addVar(vtype=gp.GRB.BINARY)
    for i in D for j in D for k in D
}

MinDigitCount = {
    d:
    m.addConstr(
        gp.quicksum(k.count(d) * X[k] for k in X) >= 1
    )
    for d in D
}

MaxDigitCount = {
    d:
    m.addConstr(
        gp.quicksum(k.count(d) * X[k] for k in X) <= math.ceil(n/3)
    )
    for d in D
}

SumToOne = m.addConstr(
    gp.quicksum(
        X[k] * k[0] / (10*k[1] + k[2]) 
        for k in X
    ) == 1
)

UseNTerms = m.addConstr(gp.quicksum(X.values()) == n)


def Callback(model, where):
    if where == gp.GRB.Callback.MIPSOL:
        XV = model.cbGetSolution(X)
        Total = 0
        for k in X:
            if round(XV[k]) > 0:
                Total += round(XV[k]) * (fractions.Fraction(k[0], k[1] * 10 + k[2]))

        # If the solution is not exact, CUT IT OFF!
        if Total != 1:
            # Use some unused X
            model.cbLazy(gp.quicksum(X[k] for k in X if round(XV[k]) == 0) >= 1)

m.Params.LazyConstraints = 1

m.optimize(Callback)




Total = 0
for k in X:
    if round(X[k].x) > 0:
        print(f"{round(X[k].x)}, {k}")
        Total += round(X[k].x) * (fractions.Fraction(k[0],10*k[1] + k[2]))
print(Total)

Set parameter LazyConstraints to value 1
Gurobi Optimizer version 12.0.3 build v12.0.3rc0 (mac64[arm] - Darwin 24.6.0 24G84)

CPU model: Apple M4 Pro
Thread count: 12 physical cores, 12 logical processors, using up to 12 threads

Non-default parameters:
LazyConstraints  1

Optimize a model with 20 rows, 729 columns and 5364 nonzeros
Model fingerprint: 0xcde94f1a
Variable types: 0 continuous, 729 integer (729 binary)
Coefficient statistics:
  Matrix range     [1e-02, 3e+00]
  Objective range  [0e+00, 0e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+01]
Presolve removed 0 rows and 1 columns
Presolve time: 0.00s
Presolved: 20 rows, 728 columns, 5357 nonzeros
Variable types: 0 continuous, 728 integer (728 binary)

Root relaxation: objective 0.000000e+00, 30 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    0.0

AttributeError: Unable to retrieve attribute 'x'

### Objective
Minimise LCM of $x_{ijk}$ used (only care about demonitor so jk)

Add $y_{jk} \in \{0,1\}$ binary 1 if $10j + k$ used in denomiator

New constraint to enforce this

$$
\sum_{i \in D} x_{ijk} \leq y_{jk} \lceil\frac{n}{3}\rceil
$$

To work out prime factorisation introdce:

$\lambda(j,k,p)$ Power for prime p in $10j + k$ $p \in P$

$z_p \in \mathbb{I}$ such that

$z_p \geq \lambda(j,k,p) y_{ij}$ ${\forall p \in P, j \in D, k \in D}$

The LCM and thus the objective function is equal to 

$$
\min \prod_{p \in P} p^{z_p}
$$

$$
\min \sum_{p \in P} z_p \log(p)
$$

In [10]:
import gurobipy as gp
import math
import fractions

P = [2, 3, 5, 7]
P += [i for i in range(11, 100) if not any(i % p == 0 for p in P)]

n = 20
# Digits
D = range(1, 10)

m = gp.Model()

X = {
    (i, j, k):
    m.addVar(vtype=gp.GRB.BINARY)
    for i in D for j in D for k in D
}

Y = {
    (j, k):
    m.addVar(vtype=gp.GRB.BINARY)
    for j in D for k in D
}

Z = {
    p:
    m.addVar()
    for p in P
}

XtoY = {
    (j, k):
    m.addConstr(gp.quicksum(X[i, j, k] for i in D) <= Y[j, k] * math.ceil(n/3))
    for (j, k) in Y
}


def Gamma(p, t):
    """Return the number of times p divides into t"""
    if t % p == 0:
        return 1 + Gamma(p, t // p)
    else:
        return 0 

YtoZ = {
    (p, j, k):
    m.addConstr(Z[p] >= Gamma(p, 10*j + k) * Y[j, k])
    for p in P for (j, k) in Y
}

MinDigitCount = {
    d:
    m.addConstr(
        gp.quicksum(k.count(d) * X[k] for k in X) >= 1
    )
    for d in D
}

MaxDigitCount = {
    d:
    m.addConstr(
        gp.quicksum(k.count(d) * X[k] for k in X) <= math.ceil(n/3)
    )
    for d in D
}

SumToOne = m.addConstr(
    gp.quicksum(
        X[k] * k[0] / (10*k[1] + k[2]) 
        for k in X
    ) == 1
)

UseNTerms = m.addConstr(gp.quicksum(X.values()) == n)


m.setObjective(
    gp.quicksum(math.log(p) * Z[p] for p in P),
    gp.GRB.MINIMIZE
)



def Callback(model, where):
    if where == gp.GRB.Callback.MIPSOL:
        XV = model.cbGetSolution(X)
        Total = 0
        for k in X:
            if round(XV[k]) > 0:
                Total += round(XV[k]) * (fractions.Fraction(k[0], k[1] * 10 + k[2]))

        # If the solution is not exact, CUT IT OFF!
        if Total != 1:
            # Use some unused X
            model.cbLazy(gp.quicksum(X[k] for k in X if round(XV[k]) == 0) >= 1)

m.Params.LazyConstraints = 1

m.optimize(Callback)




Total = 0
for k in X:
    if round(X[k].x) > 0:
        print(f"{round(X[k].x)}, {k}")
        Total += round(X[k].x) * (fractions.Fraction(k[0],10*k[1] + k[2]))
print(Total)

Set parameter LazyConstraints to value 1
Gurobi Optimizer version 12.0.3 build v12.0.3rc0 (mac64[arm] - Darwin 24.6.0 24G84)

CPU model: Apple M4 Pro
Thread count: 12 physical cores, 12 logical processors, using up to 12 threads

Non-default parameters:
LazyConstraints  1

Optimize a model with 2126 rows, 835 columns and 8337 nonzeros
Model fingerprint: 0xad2f9d96
Variable types: 25 continuous, 810 integer (810 binary)
Coefficient statistics:
  Matrix range     [1e-02, 7e+00]
  Objective range  [7e-01, 5e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+01]
Presolve removed 1887 rows and 1 columns
Presolve time: 0.01s
Presolved: 239 rows, 834 columns, 6442 nonzeros
Variable types: 25 continuous, 809 integer (809 binary)

Root relaxation: objective 1.715123e+00, 807 iterations, 0.01 seconds (0.01 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0    